# Алгоритмы сегментации (Watershed, GrabCut, Threshold)

In [ ]:
import cv2
import numpy as np
import matplotlib.pyplot as plt
from IPython.display import Image, display
from scipy import ndimage
import time
import os

In [ ]:
# ===============================
# Создание тестовых изображений
# ===============================

def create_watershed_test_image():
    """Создаёт изображение с касающимися объектами для Watershed"""

    print("📝 Создаём изображение для Watershed...")

    # Фон
    img = np.ones((400, 600, 3), dtype=np.uint8) * 200

    # Несколько "монет" (белых кругов) на сером фоне
    cv2.circle(img, (100, 100), 40, (255, 255, 255), -1)
    cv2.circle(img, (150, 120), 40, (255, 255, 255), -1)
    cv2.circle(img, (200, 90), 40, (255, 255, 255), -1)

    cv2.circle(img, (350, 150), 50, (255, 255, 255), -1)
    cv2.circle(img, (420, 180), 45, (255, 255, 255), -1)
    cv2.circle(img, (480, 160), 48, (255, 255, 255), -1)

    cv2.circle(img, (150, 300), 50, (255, 255, 255), -1)
    cv2.circle(img, (250, 280), 48, (255, 255, 255), -1)
    cv2.circle(img, (350, 320), 52, (255, 255, 255), -1)

    # Добавляем шум
    noise = np.random.randint(-15, 15, img.shape, dtype=np.int16)
    img = np.clip(img.astype(np.int16) + noise, 0, 255).astype(np.uint8)

    return img

def create_grabcut_test_image():
    """Создаёт изображение для GrabCut (объект на фоне)"""

    print("📝 Создаём изображение для GrabCut...")

    # Фон - зелёный природный
    img = np.ones((400, 600, 3), dtype=np.uint8) * 100
    img[:, :] = [100, 150, 100]  # Зелёный фон

    # Добавляем текстуру фона
    for _ in range(100):
        x, y = np.random.randint(0, 600), np.random.randint(0, 400)
        cv2.circle(img, (x, y), np.random.randint(1, 5),
                  (np.random.randint(50, 150), np.random.randint(100, 200),
                   np.random.randint(50, 150)), -1)

    # Объект в центре (красная машина - прямоугольник)
    cv2.rectangle(img, (150, 150), (450, 300), (50, 100, 200), -1)

    # Окна
    cv2.rectangle(img, (180, 170), (220, 210), (150, 150, 150), -1)
    cv2.rectangle(img, (390, 170), (430, 210), (150, 150, 150), -1)

    # Добавляем шум
    noise = np.random.randint(-10, 10, img.shape, dtype=np.int16)
    img = np.clip(img.astype(np.int16) + noise, 0, 255).astype(np.uint8)

    return img

# Создаём изображения
watershed_img = create_watershed_test_image()
grabcut_img = create_grabcut_test_image()

fig, axes = plt.subplots(1, 2, figsize=(14, 5))

axes[0].imshow(cv2.cvtColor(watershed_img, cv2.COLOR_BGR2RGB))
axes[0].set_title('Изображение для Watershed\n(касающиеся объекты)')
axes[0].axis('off')

axes[1].imshow(cv2.cvtColor(grabcut_img, cv2.COLOR_BGR2RGB))
axes[1].set_title('Изображение для GrabCut\n(объект на фоне)')
axes[1].axis('off')

plt.tight_layout()
plt.show()

In [ ]:
# ===============================
# ЗАДАНИЕ 1: Watershed алгоритм
# ===============================

def watershed_segmentation(img):
    """
    Watershed сегментация для разделения касающихся объектов

    Args:
        img: исходное изображение (BGR)

    Returns:
        segmented: сегментированное изображение с контурами
        markers: матрица маркеров
    """

    print("🔄 Выполняем Watershed сегментацию...")

    # 1. Преобразуем в grayscale
    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)

    # 2. Бинаризация
    ret, binary = cv2.threshold(gray, 0, 255, cv2.THRESH_BINARY + cv2.THRESH_OTSU)

    print("   ✓ Бинаризация завершена")

    # 3. Морфологические операции для очистки
    kernel = cv2.getStructuringElement(cv2.MORPH_ELLIPSE, (5, 5))

    # Открытие (удаление шума)
    binary = cv2.morphologyEx(binary, cv2.MORPH_OPEN, kernel, iterations=2)

    # Закрытие (заполнение дыр)
    binary = cv2.morphologyEx(binary, cv2.MORPH_CLOSE, kernel, iterations=2)

    print("   ✓ Морфологические операции завершены")

    # 4. Нахождение локальных минимумов (маркеров)
    # Используем дистанционное преобразование
    dist_transform = cv2.distanceTransform(binary, cv2.DIST_L2, cv2.DIST_MASK_PRECISE)

    # Нормализуем для визуализации
    dist_norm = cv2.normalize(dist_transform, None, 0, 255, cv2.NORM_MINMAX).astype(np.uint8)

    # Нахождение локальных максимумов дистанс-трансформа
    ret, sure_fg = cv2.threshold(dist_transform, 0.7*dist_transform.max(), 255, 0)

    # Находим коннектированные компоненты = маркеры
    ret, markers = cv2.connectedComponents(np.uint8(sure_fg))

    print(f"   ✓ Найдено {ret-1} маркеров")

    # 5. Применяем Watershed
    markers = cv2.watershed(img, markers)

    # 6. Визуализируем результаты
    # Границы Watershed = места, где markers == -1
    img_copy = img.copy()
    img_copy[markers == -1] = [0, 0, 255]  # Красные границы

    print("   ✓ Watershed завершён")

    return img_copy, markers, dist_norm, sure_fg, binary

# Применяем Watershed
segmented_watershed, markers, dist_norm, sure_fg, binary_clean = watershed_segmentation(watershed_img)

# Раскрашиваем маркеры разными цветами для наглядности
def colorize_markers(markers):
    """Раскрашивает маркеры в разные цвета"""

    colored = np.zeros((*markers.shape, 3), dtype=np.uint8)

    # Цвета для маркеров
    colors = [
        [0, 0, 0],        # Фон - чёрный
        [255, 0, 0],      # Маркер 1 - красный
        [0, 255, 0],      # Маркер 2 - зелёный
        [0, 0, 255],      # Маркер 3 - синий
        [255, 255, 0],    # Маркер 4 - голубой
        [255, 0, 255],    # Маркер 5 - пурпурный
        [0, 255, 255],    # Маркер 6 - жёлтый
    ]

    for i in range(1, len(colors)):
        mask = markers == i
        if np.any(mask):
            colored[mask] = colors[i % len(colors)]

    # Границы - белые
    colored[markers == -1] = [255, 255, 255]

    return colored

markers_colored = colorize_markers(markers)

# Визуализируем все этапы
fig, axes = plt.subplots(2, 3, figsize=(18, 10))

axes[0, 0].imshow(cv2.cvtColor(watershed_img, cv2.COLOR_BGR2RGB))
axes[0, 0].set_title('Исходное изображение')
axes[0, 0].axis('off')

axes[0, 1].imshow(binary_clean, cmap='gray')
axes[0, 1].set_title('После бинаризации\nи морфологических операций')
axes[0, 1].axis('off')

axes[0, 2].imshow(dist_norm, cmap='hot')
axes[0, 2].set_title('Дистанционное преобразование')
axes[0, 2].axis('off')

axes[1, 0].imshow(sure_fg, cmap='gray')
axes[1, 0].set_title('Маркеры (локальные максимумы)')
axes[1, 0].axis('off')

axes[1, 1].imshow(markers_colored)
axes[1, 1].set_title('Маркеры (раскрашено)')
axes[1, 1].axis('off')

axes[1, 2].imshow(cv2.cvtColor(segmented_watershed, cv2.COLOR_BGR2RGB))
axes[1, 2].set_title('Результат Watershed\n(красные границы)')
axes[1, 2].axis('off')

plt.suptitle('Пошаговое выполнение Watershed алгоритма', fontsize=14)
plt.tight_layout()
plt.show()

# Подсчитываем найденные объекты
n_objects = len(np.unique(markers)) - 2  # Минус фон и граница
print(f"\n📊 Найдено объектов: {n_objects}")

In [ ]:
# ===============================
# ЗАДАНИЕ 2: GrabCut интерактивная сегментация
# ===============================

def grabcut_segmentation(img, rect, iterations=5):
    """
    GrabCut сегментация

    Args:
        img: исходное изображение (BGR)
        rect: (x, y, width, height) - ограничивающий прямоугольник
        iterations: количество итераций алгоритма

    Returns:
        mask: маска сегментации
        output: результирующее изображение
    """

    print(f"🔄 Выполняем GrabCut с {iterations} итерациями...")

    # Создаём модели для фона и объекта
    bgdModel = np.zeros((1, 65), np.float64)
    fgdModel = np.zeros((1, 65), np.float64)

    # Маска инициализации
    mask = np.zeros(img.shape[:2], np.uint8)

    # Применяем GrabCut
    cv2.grabCut(img, mask, rect, bgdModel, fgdModel, iterations, cv2.GC_INIT_WITH_RECT)

    # Маска содержит: 0=BGD, 1=FGD, 2=PR_BGD, 3=PR_FGD
    # Объединяем вероятные и определённые переди-планы
    mask2 = np.where((mask == 2) | (mask == 0), 0, 1).astype('uint8')

    # Применяем маску к изображению
    output = img.copy()
    output[mask2 == 0] = 0  # Фон становится чёрным

    print("   ✓ GrabCut завершён")

    return mask, mask2, output, bgdModel, fgdModel

# Определяем ограничивающий прямоугольник (вручную для демонстрации)
rect = (100, 100, 400, 200)  # (x, y, width, height)

# Применяем GrabCut
mask_raw, mask_binary, grabcut_output, bgd, fgd = grabcut_segmentation(grabcut_img, rect, iterations=5)

# Визуализируем процесс
fig, axes = plt.subplots(2, 2, figsize=(14, 10))

# Исходное изображение с ограничивающим прямоугольником
img_with_rect = grabcut_img.copy()
x, y, w, h = rect
cv2.rectangle(img_with_rect, (x, y), (x+w, y+h), (0, 255, 255), 3)

axes[0, 0].imshow(cv2.cvtColor(img_with_rect, cv2.COLOR_BGR2RGB))
axes[0, 0].set_title('Исходное изображение\nс ограничивающим прямоугольником')
axes[0, 0].axis('off')

# Маска GrabCut (4 класса)
mask_viz = np.zeros((*mask_raw.shape, 3), dtype=np.uint8)
mask_viz[mask_raw == 0] = [0, 0, 0]       # Определённо фон - чёрный
mask_viz[mask_raw == 1] = [0, 255, 0]     # Определённо объект - зелёный
mask_viz[mask_raw == 2] = [128, 0, 0]     # Вероятно фон - тёмно-красный
mask_viz[mask_raw == 3] = [0, 128, 0]     # Вероятно объект - тёмно-зелёный

axes[0, 1].imshow(mask_viz)
axes[0, 1].set_title('GrabCut маска (4 класса)\n0-чёр, 1-зел, 2-тёмн-крас, 3-тёмн-зел')
axes[0, 1].axis('off')

# Бинарная маска
axes[1, 0].imshow(mask_binary, cmap='gray')
axes[1, 0].set_title('Бинарная маска\n(чёрный=фон, белый=объект)')
axes[1, 0].axis('off')

# Результат
axes[1, 1].imshow(cv2.cvtColor(grabcut_output, cv2.COLOR_BGR2RGB))
axes[1, 1].set_title('Сегментированное изображение\n(чёрный фон удалён)')
axes[1, 1].axis('off')

plt.suptitle('GrabCut интерактивная сегментация', fontsize=14)
plt.tight_layout()
plt.show()

In [ ]:
# ===============================
# ЗАДАНИЕ 3: Пороговая фильтрация
# ===============================

print("\n📋 ЗАДАНИЕ 3: Пороговая фильтрация")

# Создаём изображение для демонстрации пороговой фильтрации
threshold_img = watershed_img.copy()
gray_threshold = cv2.cvtColor(threshold_img, cv2.COLOR_BGR2GRAY)

# Применяем разные методы пороговой фильтрации

# 1. Бинарная пороговая фильтрация с разными порогами
thresholds = [50, 100, 150, 200]
fig, axes = plt.subplots(2, 2, figsize=(12, 10))

for i, threshold in enumerate(thresholds):
    ret, binary = cv2.threshold(gray_threshold, threshold, 255, cv2.THRESH_BINARY)

    axes[i // 2, i % 2].imshow(binary, cmap='gray')
    axes[i // 2, i % 2].set_title(f'Порог = {threshold}')
    axes[i // 2, i % 2].axis('off')

plt.suptitle('Бинарная пороговая фильтрация с разными порогами', fontsize=14)
plt.tight_layout()
plt.show()

# 2. Автоматическая пороговая фильтрация Otsu
ret_otsu, binary_otsu = cv2.threshold(gray_threshold, 0, 255, cv2.THRESH_BINARY + cv2.THRESH_OTSU)

# 3. Адаптивная пороговая фильтрация
adaptive_mean = cv2.adaptiveThreshold(gray_threshold, 255, cv2.ADAPTIVE_THRESH_MEAN_C,
                                      cv2.THRESH_BINARY, 11, 2)
adaptive_gaussian = cv2.adaptiveThreshold(gray_threshold, 255, cv2.ADAPTIVE_THRESH_GAUSSIAN_C,
                                          cv2.THRESH_BINARY, 11, 2)

# Визуализируем методы пороговой фильтрации
fig, axes = plt.subplots(2, 3, figsize=(16, 10))

axes[0, 0].imshow(gray_threshold, cmap='gray')
axes[0, 0].set_title('Исходное (grayscale)')
axes[0, 0].axis('off')

axes[0, 1].imshow(binary_otsu, cmap='gray')
axes[0, 1].set_title(f'Otsu (автоматический порог = {ret_otsu})')
axes[0, 1].axis('off')

axes[0, 2].imshow(adaptive_mean, cmap='gray')
axes[0, 2].set_title('Адаптивная (среднее)')
axes[0, 2].axis('off')

# Гистограмма
axes[1, 0].hist(gray_threshold.flatten(), bins=256, range=[0, 256])
axes[1, 0].axvline(ret_otsu, color='red', linestyle='--', label=f'Otsu = {ret_otsu}')
axes[1, 0].set_xlabel('Интенсивность')
axes[1, 0].set_ylabel('Количество пикселей')
axes[1, 0].set_title('Гистограмма с порогом Otsu')
axes[1, 0].legend()
axes[1, 0].grid(True, alpha=0.3)

axes[1, 1].imshow(adaptive_gaussian, cmap='gray')
axes[1, 1].set_title('Адаптивная (гауссово)')
axes[1, 1].axis('off')

# Дополнительная демонстрация: неравномерное освещение
# Создаём изображение с неравномерным освещением
non_uniform = np.zeros((400, 600), dtype=np.uint8)
for i in range(400):
    for j in range(600):
        # Левая часть темнее
        intensity = int(150 + j * 0.2)
        non_uniform[i, j] = np.clip(intensity, 0, 255)

# Добавляем объект (светлый квадрат)
non_uniform[150:250, 200:400] = 200

# Простой порог
ret_simple, binary_simple = cv2.threshold(non_uniform, 150, 255, cv2.THRESH_BINARY)

# Адаптивный порог
adaptive_non_uniform = cv2.adaptiveThreshold(non_uniform, 255, cv2.ADAPTIVE_THRESH_GAUSSIAN_C,
                                             cv2.THRESH_BINARY, 31, 5)

axes[1, 2].imshow(non_uniform, cmap='gray')
axes[1, 2].set_title('Неравномерное освещение')
axes[1, 2].axis('off')

plt.suptitle('Сравнение методов пороговой фильтрации', fontsize=14)
plt.tight_layout()
plt.show()

# Показываем эффект неравномерного освещения
fig, axes = plt.subplots(1, 3, figsize=(16, 5))

axes[0].imshow(non_uniform, cmap='gray')
axes[0].set_title('Исходное (неравномерное освещение)')
axes[0].axis('off')

axes[1].imshow(binary_simple, cmap='gray')
axes[1].set_title('Простой порог (не работает)')
axes[1].axis('off')

axes[2].imshow(adaptive_non_uniform, cmap='gray')
axes[2].set_title('Адаптивный порог (работает хорошо)')
axes[2].axis('off')

plt.suptitle('Пороговая фильтрация при неравномерном освещении', fontsize=14)
plt.tight_layout()
plt.show()

# СРАВНЕНИЕ МЕТОДОВ

Watershed:
- ✓ Хорош для: разделения касающихся объектов
- ✗ Минусы: over-segmentation, чувствителен к шуму

GrabCut:
- ✓ Хорош для: интерактивное выделение объекта из фона
- ✗ Минусы: требует взаимодействия пользователя

Пороговая фильтрация:
- ✓ Хорош для: простые, контрастные изображения
- ✗ Минусы: плохо с неравномерным освещением


# ДОМАШНЕЕ ЗАДАНИЕ

1. Марко-управляемая сегментация:
- Реализуйте возможность установки маркеров вручную
- Используйте их в Watershed

2. Улучшение GrabCut:
- Добавьте возможность рисовать штрихи для уточнения
- Реализуйте итеративное уточнение

3. Комбинированный пайплайн:
- Предварительная обработка (фильтр, морфология)
- Пороговая фильтрация
- Watershed для разделения
- Постобработка (удаление малых объектов, слияние)
